In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./5.auth.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Part No'] = input_['Part No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests

        import sys
        sys.path.append('../00.Tools')
        from crawler_configuration import get_header, get_proxy

        try:
            b = 0
            while True:

                while True:
                    b += 1
                    try:
                        resp = requests.get(f'''https://productdesk-api.cellacore.net/legacy/cardone/FinishedGood/?q=productDeskItemNumber.keyword:{input_.loc[a, 'Part Number']}&size=1&useCache=true&auth={input_.loc[a, 'Auth']}''',
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
                        break
                    except:
                        continue

                if resp.status_code == 200:
                    break

            # = = = = = = = = = = = = = = =

            json_ = resp.json()

            # = = = = = = = = = = = = = = =

            list_oem = []
            list_dict = json_['CrossReferences'] if 'CrossReferences' in json_ else []
            for dict_ in list_dict:
                if dict_['referenceType'].strip() == 'OE Part':
                    list_oem.append(dict_['reference'])

            oem = ';'.join(sorted(list(set(list_oem)), key=list_oem.index))

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame([{'Part No': input_.loc[a, 'Part No'],
                                     'Title': input_.loc[a, 'Title'],
                                     'Subtitle': input_.loc[a, 'Subtitle'],
                                     'Part Number': input_.loc[a, 'Part Number'],
                                     'Part Type': input_.loc[a, 'Part Type'],
                                     'Json_Application_Summary': input_.loc[a, 'Json_Application_Summary'],
                                     'Bundle': input_.loc[a, 'Bundle'],
                                     'Auth': input_.loc[a, 'Auth'],
                                     'OEM': oem,
                                     'Json_Info': input_.loc[a, 'Json_Info'],
                                     'Url': input_.loc[a, 'Url'],
                                     'Json_Src': input_.loc[a, 'Json_Src']}])

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'Part No': input_.loc[a, 'Part No'],
                                     'Title': input_.loc[a, 'Title'],
                                     'Subtitle': input_.loc[a, 'Subtitle'],
                                     'Part Number': input_.loc[a, 'Part Number'],
                                     'Part Type': input_.loc[a, 'Part Type'],
                                     'Json_Application_Summary': input_.loc[a, 'Json_Application_Summary'],
                                     'Bundle': input_.loc[a, 'Bundle'],
                                     'Auth': input_.loc[a, 'Auth'],
                                     'Json_Info': input_.loc[a, 'Json_Info'],
                                     'Url': input_.loc[a, 'Url'],
                                     'Json_Src': input_.loc[a, 'Json_Src']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b:}] - {input_.loc[a, 'Part No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Part No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./6.OEM-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Part No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./oem_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：9] - 37246.43-1164
[剩余数量：1000] - [当前时间：02:58:40]

[状态：ok，尝试次数：5] - 37247.43-1165
[剩余数量：999] - [当前时间：02:58:40]

[状态：ok，尝试次数：6] - 37250.43-1169
[剩余数量：998] - [当前时间：02:58:41]

[状态：ok，尝试次数：12] - 37251.43-1170
[剩余数量：997] - [当前时间：02:58:41]

[状态：ok，尝试次数：1] - 37253.43-1172
[剩余数量：996] - [当前时间：02:58:41]

[状态：ok，尝试次数：5] - 37254.43-1174
[剩余数量：995] - [当前时间：02:58:42]

[状态：ok，尝试次数：14] - 37214.40-458
[剩余数量：994] - [当前时间：02:58:42]

[状态：ok，尝试次数：8] - 37257.43-1207
[剩余数量：993] - [当前时间：02:58:42]

[状态：ok，尝试次数：34] - 37215.40-460
[剩余数量：992] - [当前时间：02:58:42]

[状态：ok，尝试次数：5] - 37224.43-1015
[剩余数量：991] - [当前时间：02:58:43]

[状态：ok，尝试次数：24] - 37252.43-1171
[剩余数量：990] - [当前时间：02:58:43]

[状态：ok，尝试次数：7] - 37261.43-1229
[剩余数量：989] - [当前时间：02:58:43]

[状态：ok，尝试次数：22] - 37256.43-1202
[剩余数量：988] - [当前时间：02:58:44]

[状态：ok，尝试次数：16] - 37259.43-1225
[剩余数量：987] - [当前时间：02:58:44]

[状态：ok，尝试次数：14] - 37260.43-1227
[剩余数量：986] - [当前时间：02:58:44]

[状态：ok，尝试次数：43] - 37183.40-387
[剩余数量：985] - [当前时间：02:58:45]

[状态：ok，尝试次数：3] - 3